In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.stats import bernoulli
import imageio
import random
from os import path as path
from itertools import islice

In [169]:
#use thresholds for theta = (0.05,0.2) and alpha=1

thresholds =  (-2.0000000000000426, 1.8799999999998747)
theta = (0.05,0.2) #absent, present
LLRs = (-0.17185025692665915,1.3862943611198906) #absent, present

def run_trial_RC(present,noise_sigma):
    
    
    decision = np.nan
    RT = 0
    LLR = 0
    samples = []
    evidence = []
    
    
    while np.isnan(decision):
        
        bias = min(max(norm(theta[present],noise_sigma).rvs(),0),1)
        #evidence is sampled according to the true distribution
        sample = bernoulli(bias).rvs()
        
        evidence.append(bias)
        samples.append(sample)
        
        #beliefs are updated according to the agent's beliefs about the distribution
        LLR += LLRs[sample]
        
        if LLR>=thresholds[1]:
            decision=1
        elif LLR<=thresholds[0]:
            decision=0
        RT += 1
        
    #post decisional evidence before extracting confidence            
    for _ in range(0):
        bias = min(max(norm(theta[present],noise_sigma).rvs(),0),1)
        #evidence is sampled according to the true distribution
        sample = bernoulli(bias).rvs()
        
        evidence.append(bias)
        samples.append(sample)
        
        #beliefs are updated according to the agent's beliefs about the distribution
        LLR += LLRS[sample]
        
    if decision==1:
        confidence = np.exp(LLR)/(1+np.exp(LLR))
    elif decision==0:
        confidence = 1/(1+np.exp(LLR))

        
    return((decision,RT,confidence,samples,evidence))

In [179]:
simulated_trials_RC_df = pd.DataFrame(columns=['present','trial','frame','decision','RT','confidence','sample','evidence'])

num_trials = 10000

noise_sd = 0.05

random.seed(1)

# Assuming `simulated_trials_RC_df` is already defined and has the correct columns
# If not, initialize it like so:
# simulated_trials_RC_df = pd.DataFrame(columns=['target_present', 'trial', 'frame', 'decision', 'RT', 'confidence', 'samples', 'evidence'])

for target_present in [1, 0]:
    for trial in range(num_trials):
        decision, RT, confidence, samples, evidence = run_trial_RC(target_present, noise_sd)
        n = len(evidence)
        frame = range(n)
        
        # Create a dictionary where scalar values are broadcasted to the length of `n` by repeating them
        data = {
            'present': [target_present] * n, 
            'trial': [trial] * n,
            'frame': frame,
            'decision': [decision] * n,
            'RT': [RT] * n,
            'confidence': [confidence] * n,
            'sample': samples,
            'evidence': evidence
        }
        
        # Convert the dictionary to a DataFrame
        trial_df = pd.DataFrame(data)
        
        # Append the DataFrame for this trial to the main DataFrame
        simulated_trials_RC_df = pd.concat([simulated_trials_RC_df, trial_df], ignore_index=True)

simulated_trials_RC_df.to_csv('simulated_data/accum_bernoulli_RC.csv', index=False)


In [178]:
simulated_trials_RC_df

,present,trial,frame,decision,RT,confidence,sample,evidence
0,1,0,0,1,7,0.871394,0,0.178330
1,1,0,1,1,7,0.871394,0,0.178360
2,1,0,2,1,7,0.871394,0,0.229289
3,1,0,3,1,7,0.871394,0,0.197038
4,1,0,4,1,7,0.871394,1,0.190703
...,...,...,...,...,...,...,...,...
313300,0,9998,20,0,21,0.886019,0,0.040094
313301,0,9999,0,1,4,0.919004,0,0.051587
313302,0,9999,1,1,4,0.919004,1,0.061036
313303,0,9999,2,1,4,0.919004,0,0.061294
